In [121]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [122]:
tsla = yf.Ticker("tsla")

tsla_hist = tsla.history(period='5y', interval='1d', end='2025-06-13')
# tsla_hist

In [123]:
tsla_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [124]:
tsla_hist.index =pd.to_numeric(tsla_hist.index)
tsla_hist.index = tsla_hist.index/(max(tsla_hist.index)) + 1

In [125]:
for col in tsla_hist.columns:
    tsla_hist[col] = tsla_hist[col]/max(tsla_hist[col]) + 1

In [126]:
X = tsla_hist.drop(columns=['Close'])
y = tsla_hist['Close']

In [127]:
X['Date'] = X.index

In [128]:
for delay in range(1,4):
    X[f"Delay {delay}"] = y.iloc[(3-delay):-(delay)]

In [129]:
X_sample = [[d, h, l, o, vol, v1, v2, v3] for d, h, l, o, vol, v1, v2, v3 in zip(X['Date'].iloc[3:].values, X['High'].iloc[3:].values, X['Low'].iloc[3:].values,X['Open'].iloc[3:].values, X['Volume'].iloc[3:].values, X['Delay 1'].dropna().values, X['Delay 2'].dropna().values, X['Delay 3'].dropna().values)]
y_sample = y.iloc[3:].values

In [132]:
len(X['Date'].iloc[3:].values), len(X_sample)

(1252, 1252)

In [135]:
len(y_sample), len(X_sample)

(1252, 1252)

In [136]:
torch.set_default_dtype(torch.float64)

In [137]:
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))

In [138]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [139]:
len(X_test)

251

In [140]:
train_data = [(X,y) for X, y in zip(X[:-273], y[:-273])]
test_data = [(X,y) for X, y in zip(X[-273:], y[-273:])]

In [141]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=32,)
test_dataloader= DataLoader(test_data, batch_size=32,)

In [149]:
count = 0
batch = 0
for (X, y) in train_dataloader:
    if (torch.any(X.isnan())):
        print(f"Found on batch: {batch}")
        print(X)
        count+=1
    batch +=1


Found on batch: 0
tensor([[1.1401, 1.1377, 1.1443, 1.1432, 1.9101,    nan,    nan, 1.1381],
        [1.1399, 1.1381, 1.1448, 1.1433, 1.9101,    nan, 1.1392, 1.1392],
        [1.1393, 1.1366, 1.1389, 1.2467, 1.9102, 1.1335, 1.1335, 1.1335],
        [1.1337, 1.1345, 1.1366, 1.2083, 1.9102, 1.1370, 1.1370, 1.1370],
        [1.1394, 1.1358, 1.1391, 1.1993, 1.9103, 1.1333, 1.1333, 1.1333],
        [1.1357, 1.1378, 1.1382, 1.2032, 1.9104, 1.1402, 1.1402, 1.1402],
        [1.1410, 1.1484, 1.1463, 1.3808, 1.9105, 1.1500, 1.1500, 1.1500],
        [1.1517, 1.1549, 1.1574, 1.3000, 1.9105, 1.1555, 1.1555, 1.1555],
        [1.1711, 1.1676, 1.1728, 1.3883, 1.9106, 1.1679, 1.1679, 1.1679],
        [1.1788, 1.1880, 1.1845, 1.4630, 1.9107, 1.1906, 1.1906, 1.1906],
        [1.1968, 1.1951, 1.1948, 1.4837, 1.9108, 1.1931, 1.1931, 1.1931],
        [1.1968, 1.1934, 1.1911, 1.3672, 1.9108, 1.1898, 1.1898, 1.1898],
        [1.1957, 1.1922, 1.1969, 1.2638, 1.9109, 1.1937, 1.1937, 1.1937],
        [1.1956, 1.2

In [143]:
class MLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            nn.Linear(hidden_features, 15),
            nn.ReLU()
        )
        self.l3 = nn.Linear(15, output_features)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [144]:
mlpmodel = MLP(8, 30, 1)

In [145]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(params=mlpmodel.parameters(), lr=1.e-6)

In [146]:
EPOCHS = 1

mlpmodel.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        preds = mlpmodel(X)
        # print(f"Preds + {preds}")
        loss = loss_fn(torch.unsqueeze(y, 1), preds)
        total_loss+= loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"We're on batch: {batch} + Loss: {total_loss/batch}")


We're on batch: 0 + Loss: nan
We're on batch: 1 + Loss: nan
We're on batch: 2 + Loss: nan
We're on batch: 3 + Loss: nan
We're on batch: 4 + Loss: nan
We're on batch: 5 + Loss: nan
We're on batch: 6 + Loss: nan
We're on batch: 7 + Loss: nan
We're on batch: 8 + Loss: nan
We're on batch: 9 + Loss: nan
We're on batch: 10 + Loss: nan
We're on batch: 11 + Loss: nan
We're on batch: 12 + Loss: nan
We're on batch: 13 + Loss: nan
We're on batch: 14 + Loss: nan
We're on batch: 15 + Loss: nan
We're on batch: 16 + Loss: nan
We're on batch: 17 + Loss: nan
We're on batch: 18 + Loss: nan
We're on batch: 19 + Loss: nan
We're on batch: 20 + Loss: nan
We're on batch: 21 + Loss: nan
We're on batch: 22 + Loss: nan
We're on batch: 23 + Loss: nan
We're on batch: 24 + Loss: nan
We're on batch: 25 + Loss: nan
We're on batch: 26 + Loss: nan
We're on batch: 27 + Loss: nan
We're on batch: 28 + Loss: nan
We're on batch: 29 + Loss: nan
We're on batch: 30 + Loss: nan
